In [2]:
import tensorflow as tf
import os
from tqdm import tqdm
import urllib.request as req


In [2]:
SOURCE_URL = "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip"

In [3]:
DESTINATION = "data"
os.makedirs(DESTINATION, exist_ok=True)
data_file = "data.zip"

DESTINATION_ZIP_PATH = os.path.join(DESTINATION, data_file)

In [4]:
filename, headers = req.urlretrieve(SOURCE_URL, DESTINATION_ZIP_PATH)

In [5]:
filename

'data\\data.zip'

In [7]:
print(headers)

Content-Type: application/octet-stream
Accept-Ranges: bytes
Server: Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0
x-ms-blob-content-md5: l2WJyK3tt5/f2azyQ9eE8w==
Last-Modified: Sat, 13 Oct 2018 12:14:22 GMT
ETag: "0x8D6310568FD1DF6"
Content-Length: 824894548
Date: Sun, 20 Feb 2022 11:41:56 GMT
Connection: close




In [10]:
from zipfile import ZipFile
with ZipFile(filename,'r') as zip_f:
    zip_f.extractall("data/")

In [17]:
TARGET_DATA_DIRS = ["Cat", "Dog"]
PARENT_DIR = os.path.join("data","PetImages")

In [15]:
!pip install Pillow

In [16]:
BAD_DATA_DIR = "bad_data"
os.makedirs(BAD_DATA_DIR, exist_ok=True)

In [13]:
## We are able to read the images 

img = Image.open(r"C:\Users\atulkumarrai\PycharmProjects\Ineuron practice\Ineuron_practice\CNN_Classification-main\Resource_env\data\PetImages\Dog\4.jpg")
img.format

'JPEG'

In [15]:
if img.format in ['JPEG','PNG','BMP','JPG']:
    print("format of the image is correct")

format of the image is correct


In [20]:
import shutil
import imghdr

for dirs in os.listdir(PARENT_DIR):
    full_path_data_dir= os.path.join(PARENT_DIR,dirs)
    for imgs in os.listdir(full_path_data_dir):
        path_to_img=os.path.join(full_path_data_dir, imgs)
        try:
            img=Image.open(path_to_img)
            img.verify()
            
            if len(img.getbands()) != 3 or imghdr.what(path_to_img) not in ['jpeg','png']:   ## img.getbands() will return the bands of an image, for eg- R,G,B,A
                bad_data_path=os.path.join(BAD_DATA_DIR, imgs)
                shutil.move(path_to_img,bad_data_path)
                print(f"{path_to_img} is not a verified image and has format {imghdr.what(path_to_img)}")
                continue
                
        except Exception as e:
            print(f"{path_to_img} is bad !")
            bad_data_path = os.path.join(BAD_DATA_DIR, imgs)
            shutil.move(path_to_img, bad_data_path)
                
            

data\PetImages\Cat\10125.jpg is bad !


FileNotFoundError: [Errno 2] No such file or directory: 'data\\PetImages\\Cat\\10125.jpg'

In [21]:
## Let's decide image size and batch size now
IMG_SIZE=(180,180)
BATCH_SIZE=32

In [22]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    PARENT_DIR,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    PARENT_DIR,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

Found 24998 files belonging to 2 classes.
Using 19999 files for training.
Found 24998 files belonging to 2 classes.
Using 4999 files for validation.


In [23]:
## creating logging dashboard in tensorflow
import time

def get_log_path(base_log_dir=os.path.join("logs","fit")):
    uniqueName=time.asctime().replace(" ", " _").replace(":","")
    log_path=os.path.join(base_log_dir,uniqueName)
    print(f"saving the logs at: {log_path}")
    return log_path

log_dir=get_log_path()

saving the logs at: logs\fit\Mon _Feb _21 _205818 _2022


In [24]:
file_writer = tf.summary.create_file_writer(logdir=log_dir)

In [1]:
for images, labels in train_ds.take(5):    ##train_ds.take(4) will take 4 batches of 32 images each and display it's details 
    print(images.shape, labels)   ## Shape tells us--> count=32, size=180,180 and channel=3(R,G,B)

NameError: name 'train_ds' is not defined

In [30]:
import numpy as np
with file_writer.as_default():
    images = np.array(images) ### (20, 28, 28, 1)

    tf.summary.image("samples", images.astype("uint8"), max_outputs=25, step=0)

UsageError: Line magic function `%tf` not found.
